# Training the Model

In this notebook we make use the library `objdet` to create our own model using transfer learning, and train it in Google Colab to take advantage of their RAM and GPU.

In [ ]:
import os

model_name = 'ssd_mobilenet_v2_coco_2018_03_29'

google_colab = True
if google_colab:
    root_path = '/content/drive/My Drive/Colab Notebooks'
    data_path = os.path.join(root_path, 'data', 'egohands')
else:
    root_path = '.'
    data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')

## Google Colab

Mount your drive where the original data must be. It will be later converted to TFRecords. I decide to use Drive in order to have persistent storage, so I only have to upload and generate the TFRecords once. This speeds up the training process considerably.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [ ]:
!pip install -q -r requirements.txt

In [ ]:
!ls /content/drive/'My Drive'/'Colab Notebooks'

Test if the GPU is available

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

## Install ObjDet

In [ ]:
import objdet.tfapi as tfapi

In [ ]:
tfapi.install()

## Download Model and Transfer Learning

Download the model for SSD300 on top of MobileNetV2 pretrained on MS COCO

In [ ]:
tfapi.download_model(model_name)

Generate the config files for our own model that only contains one class (i.e. hand)

In [ ]:
import os

train_steps = 40000
classes = ['hand']
tfapi.transfer_learning(train_steps, classes, model_name, data_path)

## Create TFRecords

Create the TFRecords for all images in the `data` folder. This takes time and it requires around 13GB of space in disk.

In [ ]:
import objdet.tfrecords as tfrecords

In [ ]:
classes = dict([(1, 'hand')])
tfrecords.create_tfrecords(data_path, classes)

## Tensorboard

This code has been taken from https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
!mkdir checkpoints

In [ ]:
train_dir = 'checkpoints'

In [ ]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(train_dir)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Train and Save the Model

Now we are ready to train the model

In [ ]:
tfapi.train(model_name, train_dir)

After the training we can save the latest checkpoint as the trained model that will be later be loaded for prediction

In [ ]:
tfapi.save_model(train_dir)

In [ ]:
!cp -r checkpoints /content/drive/My\ Drive/Colab\ Notebooks/